# FusionDB reactor browser

Lists all reactors side by side (one column per reactor) with every available field, including derived values. Columns with warnings are highlighted in yellow; mismatches beyond the 1% tolerance are highlighted in red (red overrides yellow). Warning messages are echoed below the table.

In [1]:
from dataclasses import fields as dataclass_fields
from pathlib import Path
import re
import warnings

import pandas as pd  # Pandas is required for styling

from fusdb.loader import find_reactor_dirs, load_reactor_yaml
from fusdb.reactors_class import Reactor

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# Load reactors and capture warnings per reactor and field
root = Path('.')
reactors = {}
cell_status: dict[tuple[str, str], str] = {}
computed_flags: dict[tuple[str, str], bool] = {}
warning_messages: dict[str, list[str]] = {}
field_patterns = [
    re.compile(r"Inconsistent values for ([A-Za-z0-9_]+)", re.IGNORECASE),
    re.compile(r"^([A-Za-z0-9_]+) differs", re.IGNORECASE),
    re.compile(r"^([A-Za-z0-9_]+) updated", re.IGNORECASE),
    re.compile(r"(density_profile_file)", re.IGNORECASE),
]

def classify_status(msg: str) -> str:
    text = msg.lower()
    if "within tolerance" in text:
        return "yellow"
    if "inconsistent" in text or "updated from" in text or "not found" in text:
        return "red"
    return "yellow"

for reactor_dir in find_reactor_dirs(root):
    path = reactor_dir / 'reactor.yaml'
    with warnings.catch_warnings(record=True) as caught:
        warnings.simplefilter('always')
        reactor = load_reactor_yaml(path)

    reactors[reactor.id] = reactor
    messages = [str(w.message) for w in caught]
    warning_messages[reactor.id] = messages
    for fname, src_name in getattr(reactor, '_sources', {}).items():
        computed_flags[(fname, reactor.id)] = src_name != 'explicit'

    for msg in messages:
        field = None
        for pat in field_patterns:
            m = pat.search(msg)
            if m:
                field = m.group(1)
                break
        if field is None:
            continue
        cell_status[(field, reactor.id)] = classify_status(msg)

# Build a DataFrame with every dataclass field as rows
field_names = [f.name for f in dataclass_fields(Reactor)]
table_data: dict[str, list[object]] = {}
for rid, reactor in sorted(reactors.items()):
    table_data[rid] = [getattr(reactor, name) for name in field_names]

df = pd.DataFrame(table_data, index=field_names)

def style_cells(data: pd.DataFrame) -> pd.DataFrame:
    styles = pd.DataFrame('', index=data.index, columns=data.columns)
    for field in styles.index:
        for rid in styles.columns:
            parts = []
            if computed_flags.get((field, rid)):
                parts.append('text-decoration: underline;')
            status = cell_status.get((field, rid))
            if status == 'red':
                parts.append('color: #b30000; font-weight: 600;')
            elif status == 'yellow':
                parts.append('color: #b58900;')
            styles.loc[field, rid] = ' '.join(parts)
    return styles

styled = df.style.apply(style_cells, axis=None)
display(styled)

# Show warnings below the table
for rid, msgs in warning_messages.items():
    if not msgs:
        continue
    print(f'Warnings for {rid}:')
    for msg in msgs:
        print(f' - {msg}')


,ARC_2015,DEMO_2018,STEP2024
id,ARC_2015,DEMO_2018,STEP2024
name,ARC 2015,EU-DEMO Physics Baseline 2018,STEP 2024
reactor_configuration,compact tokamak,tokamak,spherical tokamak
organization,MIT,EUROFUSION,UKAEA
reactor_class,ARC,DEMO,STEP
country,USA,EU,UK
site,None,None,None
design_year,2015,2018,2024
doi,10.1016/j.fusengdes.2015.07.008,10.1016/j.fusengdes.2022.113080,10.1088/1741-4326/ad6ea2
notes,Conceptual compact tokamak from MIT.,EU-DEMO Physics Baseline 2018,None


Warnings for ARC_2015:
 - A differs from Aspect ratio but is within tolerance: existing 2.92, new 2.920353982300885
 - V_p differs from Tokamak volume but is within tolerance: existing 153.0, new 153.04473400881614
 - S_p differs from Tokamak surface but is within tolerance: existing 215.23, new 215.2283581386006
 - B0 differs from Toroidal beta but is within tolerance: existing 9.2, new 9.191457954691616
 - Inconsistent tau_E: keeping 0.64 (explicit) over 1100.5075540778432 (Energy confinement time)
 - Inconsistent W_th: keeping 146849369.37092754 (Thermal stored energy) over 85400.22833023263 (Energy confinement time)
Warnings for DEMO_2018:
 - A differs from Aspect ratio but is within tolerance: existing 3.1, new 3.103448275862069
 - a differs from Tokamak volume but is within tolerance: existing 2.9, new 2.900000000000001
 - a differs from Tokamak surface but is within tolerance: existing 2.9, new 2.8999999999999995
 - P_sep_over_R differs from P_sep ratio but is within tolerance: 